# 2.3 Data Access and Basic Processing Pangeo Resample

<center><img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px; margin-left: 10px; max-height: 250px;" /></center>

<img src="https://raw.githubusercontent.com/pangeo-data/pangeo.io/refs/heads/main/public/Pangeo-assets/pangeo_logo.png"
     alt="Pangeo logo"
     style="float: center; margin-right: 10px; max-height: 80px;"/>

## Resample Operators with Pangeo ecosystem

Sometimes we need to align the spatial or temporal dimension of two datacubes, so that they can be merged together.

### Spatial resampling Sentinel-2 to match Landsat

Start importing the necessary libraries

In [ ]:
import pystac_client
import stackstac
from rasterio.enums import Resampling

Create two datacubes, one for Sentinel-2 and one for Landsat

In [ ]:
spatial_extent = [11.4, 45.5, 11.42, 45.52]
temporal_extent = ["2023-06-01", "2023-06-30"]

Datacube for Sentinel-2

In [ ]:
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)
s2_items = catalog.search(
    bbox=spatial_extent,
    datetime=temporal_extent,
    collections=["sentinel-2-l2a"]
).item_collection()

s2_cube = stackstac.stack(s2_items,
                     bounds_latlon=spatial_extent,
                     assets=["red","nir"]
)
s2_cube

Datacube for Landsat

In [ ]:
url = "https://planetarycomputer.microsoft.com/api/stac/v1"
catalog = pystac_client.Client.open(URL)
l8_items = catalog.search(
    bbox=spatial_extent,
    datetime=temporal_extent,
    collections=["landsat-c2-l2"]
).item_collection()

l8_cube = stackstac.stack(l8_items,
                     bounds_latlon=spatial_extent,
                     assets=["red","nir08"]
)
l8_cube

From the previous outputs, notice the shape difference in the spatial dimensions `x` and `y`.

This is due to the different resolution of the two collections: 10m for Sentinel-2, 30m for Landsat.

Now we use the `stackstac.stack` to load and resample the Sentinel-2 data to match Landsat.

In [ ]:
s2_cube_coarse = stackstac.stack(s2_items,
                                 bounds_latlon=spatial_extent,
                                 assets=["red","nir"],
                                 resolution=30.0, resampling=Resampling.average, epsg=32632)
s2_cube_coarse